# Documentation

**OLR Analysis**

**Spencer Ressel**

Friday, December 10th, 2021

This file does analysis on the dataset of Outgoing Longwave Radiation (OLR) from Liebmann and Smith (1994)


# Imports

In [63]:
# Data Analysis Tools
import numpy as np
import xarray as xr
from scipy import interpolate
import scipy.fft as fft
import pandas as pd
import mjo_mean_state_diagnostics as mjo

# Plotting tools
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
from matplotlib import colors
import cartopy.crs as ccrs
import cartopy.feature as cf
import cartopy.util as cutil
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter, LongitudeLocator, LatitudeLocator


# Load data

In [72]:
# Wind data
TIME_MIN = '1999-01-01T00:00:00.000000000'
TIME_MAX = '2018-12-31T00:00:00.000000000'
LAT_MIN = -30
LAT_MAX = 30

wind_data = xr.open_dataset(
    'C:/Users/resse/Desktop/Data/ECMWF/ERA5/era5_combined_wind_daily_1979_2020_pressure_levels.nc'
).sortby('lat')

upper_level_zonal_wind = wind_data['u'].sel(time=slice(TIME_MIN, TIME_MAX), lev=200, lat=slice(LAT_MIN, LAT_MAX))
lower_level_zonal_wind = wind_data['u'].sel(time=slice(TIME_MIN, TIME_MAX), lev=850, lat=slice(LAT_MIN, LAT_MAX))
upper_level_meridional_wind = wind_data['v'].sel(time=slice(TIME_MIN, TIME_MAX), lev=200, lat=slice(LAT_MIN, LAT_MAX))
lower_level_meridional_wind = wind_data['v'].sel(time=slice(TIME_MIN, TIME_MAX), lev=850, lat=slice(LAT_MIN, LAT_MAX))


olr_data = xr.open_dataset('C:/Users/resse/Desktop/Data/NOAA/olr.day.mean.nc')
outgoing_longwave_radiation = olr_data['olr'].sortby('lat').sel(time=slice(TIME_MIN, TIME_MAX), lat=slice(LAT_MIN, LAT_MAX))

outgoing_longwave_radiation_anomalies = outgoing_longwave_radiation - outgoing_longwave_radiation.mean()
lower_level_zonal_wind_anomalies = lower_level_zonal_wind - lower_level_zonal_wind.mean()
lower_level_meridional_wind_anomalies = lower_level_meridional_wind - lower_level_meridional_wind.mean()

In [80]:
plt.style.use('default')
plt.rcParams.update({'font.size':24})
cmap_modified = mjo.modified_colormap('coolwarm', 'white', 0.1, 0.1)
# cmap_modified = 'BrBG'
coastline_width = 1

gs = GridSpec(1, 2, width_ratios=(1, 0.02))
gs.update(top=.95, bottom=0.05, left=0.05, right=.95, hspace=0, wspace=0.05)
f = plt.figure(figsize=(16,4))

ax = f.add_subplot(gs[0], projection=ccrs.PlateCarree(central_longitude=-180))
cb_ax = f.add_subplot(gs[1])

# Moisture Tendency
ax.set_title(f"Time-Mean Outgoing Longwave Radiation Anomalies and 850-hPa Zonal Winds", fontsize=20)
im = ax.contourf(
    outgoing_longwave_radiation_anomalies.lon, 
    outgoing_longwave_radiation_anomalies.lat,
    outgoing_longwave_radiation_anomalies.mean(dim='time'),
    transform=ccrs.PlateCarree(), 
    # cmap='RdYlBu_r',
    cmap=cmap_modified,
    norm=mcolors.CenteredNorm(),
    levels=21
)    

cbar = fig.colorbar(im, cax=cb_ax)
cbar.ax.tick_params(labelsize=20)
cbar.set_label(r'W m$^{-2}$')

# Add cyclic point
cdata, clon = cutil.add_cyclic_point(
    outgoing_longwave_radiation_anomalies.mean(dim='time'),
    coord=outgoing_longwave_radiation_anomalies.lon
)

ax.contourf(
    clon, 
    outgoing_longwave_radiation_anomalies.lat, 
    cdata, 
    transform=ccrs.PlateCarree(), 
    # cmap='RdYlBu_r',
    cmap=cmap_modified,
    norm=mcolors.CenteredNorm(),
    levels=21
)

arrow_spacing = 1
ax.quiver(
    lower_level_zonal_wind_anomalies.lon[::2*arrow_spacing],
    lower_level_zonal_wind_anomalies.lat[::arrow_spacing],
    lower_level_zonal_wind_anomalies.mean(dim='time').values[::arrow_spacing, ::2*arrow_spacing],
    lower_level_meridional_wind_anomalies.mean(dim='time').values[::arrow_spacing, ::2*arrow_spacing],
    width=0.001,
    scale=500,
    transform=ccrs.PlateCarree(central_longitude=-180)
)

ax.set_aspect('auto')
ax.set_xlabel('')
ax.add_feature(cf.COASTLINE, lw=coastline_width)

gl = ax.gridlines(
    crs=ccrs.PlateCarree(),
    draw_labels=True,
    linewidth=1,
    color="gray",
    alpha=0.75,
    linestyle="-",
    zorder=15

)
gl.right_labels = False
gl.top_labels = False
gl.xlocator = mticker.FixedLocator(np.arange(-180,180,30))
# gl.xlocator = LongitudeLocator(30)
gl.xformatter = LongitudeFormatter()
gl.xlabel_style = {'fontsize':20}
gl.ylocator = mticker.FixedLocator(np.arange(-40,40,10))
gl.yformatter = LatitudeFormatter()
gl.ylabel_style = {'fontsize':20}

plt.show()

In [85]:
lower_level_zonal_wind_anomalies
# outgoing_longwave_radiation_anomalies

<xarray.DataArray 'u' (time: 7305, lat: 25, lon: 144)>
array([[[  4.5832825 ,   8.596289  ,  12.274662  , ...,   0.13525367,
           1.1468713 ,   3.3708856 ],
        [  4.94108   ,   8.853697  ,  11.819049  , ...,   1.6539667 ,
           2.9873464 ,   5.8317146 ],
        [  7.713375  ,   8.802216  ,  11.762419  , ...,   2.0143392 ,
           3.8445184 ,   5.739048  ],
        ...,
        [  3.5098855 ,   3.9938142 ,   6.3027744 , ...,   4.5910034 ,
          -1.5224593 ,   2.8457716 ],
        [  6.8845177 ,   8.508768  ,  11.960624  , ...,   5.62064   ,
           1.5818918 ,   1.3322065 ],
        [ 14.87192   ,  19.610819  ,  21.528515  , ...,   9.438015  ,
           8.54738   ,  11.795881  ]],

       [[  6.266737  ,  12.923333  ,  16.815357  , ...,   1.5818918 ,
           2.5188615 ,   2.9281423 ],
        [  5.7622147 ,   8.97468   ,   9.9296665 , ...,  -1.8854067 ,
           2.3000638 ,   5.401842  ],
        [  5.280861  ,   7.3401318 ,   9.5306835 , ...,  -0.8506224 ,
          -0.57262063,   2.5265844 ],
...
        [  9.414848  ,   6.282181  ,   1.1211298 , ...,  11.283638  ,
          10.331224  ,   9.878185  ],
        [ 17.27612   ,  14.753511  ,  10.086685  , ...,  20.457695  ,
          16.897728  ,  15.139625  ],
        [ 25.446281  ,  23.361267  ,  17.155136  , ...,  24.45783   ,
          24.141216  ,  25.423113  ]],

       [[ -5.401613  ,  -3.648657  ,  -3.326897  , ...,  -5.149353  ,
          -3.3758051 ,  -3.7670653 ],
        [-10.578108  , -11.015703  , -12.210081  , ..., -10.001513  ,
          -9.6153965 , -10.771166  ],
        [ -8.557447  ,  -4.88422   ,  -1.6897757 , ...,  -6.5831203 ,
          -8.531706  ,  -7.509794  ],
        ...,
        [ 11.366009  ,   8.382639  ,   3.9346101 , ...,  16.740707  ,
          15.97363   ,  15.219421  ],
        [ 19.137186  ,  15.131903  ,  13.2965765 , ...,  21.876019  ,
          25.500336  ,  22.583893  ],
        [ 23.278896  ,  23.428194  ,  21.960962  , ...,  24.480997  ,
          26.362656  ,  25.860708  ]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float32 -30.0 -27.5 -25.0 -22.5 -20.0 ... 22.5 25.0 27.5 30.0
    lev      int32 850
  * time     (time) datetime64[ns] 1999-01-01 1999-01-02 ... 2018-12-31

In [68]:
lower_level_zonal_wind_anomalies = lower_level_zonal_wind - lower_level_zonal_wind.mean()

In [70]:
lower_level_zonal_wind

<xarray.DataArray 'u' (time: 7305, lat: 0, lon: 144)>
array([], shape=(7305, 0, 144), dtype=float32)
Coordinates:
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float32 
    lev      int32 850
  * time     (time) datetime64[ns] 1999-01-01 1999-01-02 ... 2018-12-31
Attributes:
    units:          m s**-1
    long_name:      U component of wind
    standard_name:  eastward_wind